In [10]:
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting.metrics import RMSE
import pandas as pd 
import numpy as np
from pytorch_forecasting import TimeSeriesDataSet

In [15]:
db = pd.read_csv("/home/jathur/Bureau/Projects/TeleFinance/data/raw/History.csv")
db["time_idx"] = np.arange(len(db))   
uniueTicket = dbt['ticket'].unique()

In [16]:
# Paramètres de séquence
max_encoder_length = 60   # 60 jours d'historique
max_prediction_length = 30  # prédire 30 jours

In [17]:
dataset = TimeSeriesDataSet(
    db,
    time_idx="time_idx",
    target="Close",
    group_ids=["ticket"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Close", "Volume"]
)

In [18]:
train_dataloader = dataset.to_dataloader(train=True, batch_size=64, num_workers=0)

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    dataset,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,

    loss= RMSE(),
    log_interval=10,
)
trainer.fit(tft, train_dataloaders=train_dataloader)

/home/jathur/Bureau/Projects/TeleFinance/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/jathur/Bureau/Projects/TeleFinance/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [ ]:
x

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`